<a href="https://colab.research.google.com/github/vishalbalaji-v/Personal-Projects/blob/main/Guided%20Projects%20/%20Rossman%20Kaggle%3A%20Forecasting%20Sales%20/%20Part%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML-2: Trees, Model Interrogation and Bayesian Workflow
# Homework 2: Rossman Kaggle: Forecasting Sales
# Part 3 : Extracting embeddings!
**ML-2 Cohort 1** <br>
**Instructor: Dr. Rahul Dave**<br>
**Max Score: 100** <br>

In [ ]:
#importing libraries
import numpy as np
import scipy.stats
import scipy.special
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import cm
import pandas as pd
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Activation, Reshape, Flatten, Dropout, Embedding
from keras.layers import Concatenate
from keras.layers.embeddings import Embedding
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
import pickle
import csv
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

We will repeat the first initial steps again from Part 2

Lets import the feature_train_data.pickle file and set X,y values from the pickle file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
f = open('/content/drive/MyDrive/Colab files/Homework 2 - ML2/feature_train_data.pickle','rb')
(X, y) = pickle.load(f)

In [ ]:
# we will split the train_ratio and 90% and 10% and set the train_size
train_ratio = 0.9
num_records = len(X)
train_size = int(train_ratio * num_records)

In [ ]:
#lets look at our data
X[1], y[1]

(array([   0,    0,    0, 1058,    1,    0,    0,    1]), 4491)

The next set of inputs is following: 

1. Do you want to one hot encode the data?
2. Do you want to provide embeddings as input - this will be set to **True** for models with entity embeddings
3. Do you want to save the emmbeddings? - again set to **true** if you want to entity embeddings
4. if 3 is set to **true**, we want to save them to a embeddings.pickle


In [ ]:
#your code here
one_hot_as_input = False 
embeddings_as_input = True #embeddings later on needs to set to true for Part 3
save_embeddings = True

saved_embeddings_fname = "/content/drive/MyDrive/Colab files/Homework 2 - ML2/embeddings.pickle"  # set save_embeddings to True to create this file

Split the data based on the train_size - same as part 2 

In [ ]:
#your code here
X_train, X_val, y_train, y_val = train_test_split(X,y, train_size=train_ratio, random_state = 42)
print(X_train.shape, X_val.shape, len(y_train), len(y_val))

(759904, 8) (84434, 8) 759904 84434


In [ ]:
def sample(X, y, n):
  #your code here
    '''random samples'''
    num_row = X.shape[0]
    indices = np.random.randint(num_row, size=n)
    return X[indices, :], y[indices]

In [ ]:
X_train, y_train = sample(X_train, y_train, 200000)  # Simulate data sparsity
X_val, y_val = sample(X_val, y_val, 200000)  # Simulate data sparsity

## Now lets work with Models with Entity embedding!!

To read about Entity Embeddings in more detail and its use cases look at the follwing links:
1. [Enhancing categorical features with Entity Embeddings](https://towardsdatascience.com/enhancing-categorical-features-with-entity-embeddings-e6850a5e34ff)
2. [Understanding Entity Embeddings and It’s Application](https://towardsdatascience.com/understanding-entity-embeddings-and-its-application-69e37ae1501d#:~:text=Loosely%20speaking%2C%20entity%20embedding%20is,a%20sentence%2C%20or%20a%20paragraph.)

The basic outline of this model will be something like this:


![Outline.jpeg](https://drive.google.com/uc?export=view&id=1cm-jylknGEFg9KjehhRy90INkUJqRwrL)

We will be creating the following embeddings:

1. input_store - with input shape as (1,) 
  * The input_store will be passed to output_store to create an embedding layer with embedding shape as 1115, 10
  * we will reshape this to target_shape=(10,)
2. input_dow - with input shape as (1,) 
  * The input_dow will be passed to output_dow to create an embedding layer with embedding shape as 7, 6
  * we will reshape this to target_shape=(6,)
3. input_promo - with input shape as (1,) 
  * output promo will be a dense(1) for input_promo
4. input_year - with input shape as (1,) 
  * The input_year will be passed to output_year to create an embedding layer with embedding shape as 3, 2
  * we will reshape this to target_shape=(2,)
5. input_month - with input shape as (1,) 
  * The input_month will be passed to output_month to create an embedding layer with embedding shape as 12, 6
  * we will reshape this to target_shape=(6,)
6. input_day - with input shape as (1,) 
  * The input_day will be passed to output_day to create an embedding layer with embedding shape as 31, 10
  * we will reshape this to target_shape=(10,)
7. input_germanstate - with input shape as (1,) 
  * The input_germanstate will be passed to output_germanstate to create an embedding layer with embedding shape as 12, 6
  * we will reshape this to target_shape=(6,)


* Construct an input model with all the inputs 
* Construct an output embeddings with all the outputs( basically the embeddings) - concatenate this and call it the **output model**

Set the output model with the following parameters:

1. Dense Layer - 1000 neurons, keep the kernel_initializer as uniform, with activaation as relu
2. Dense Layer - 500 neurons, keep the kernel_initializer as uniform, with activaation as relu
3. Final dense layer with 1 neuron, and activation as sigmoid
4. Create a KerasModel called modelNN_emb:

       modelNN_emb = KerasModel(inputs=input_model, outputs=output_model)
4. Compile the model on mean absolute error and optimizer as adam

In [ ]:
input_store = Input(shape=(1,))
embedding_1 = Embedding(1115,10, name='store_embedding')(input_store)
output_store = Reshape(target_shape=(10,))(embedding_1)
# input_model.append(input_store)

In [ ]:
input_dow = Input(shape=(1,))
embedding_2 = Embedding(7,6, name='dow_embedding')(input_dow)
output_dow = Reshape(target_shape=(6,))(embedding_2)
# input_model.append(input_dow)

In [ ]:
input_promo = Input(shape=(1,))
# embedding_3 = Embedding(0,0, input_length=0)(input_promo)
output_promo = Dense(1)(input_promo)
# input_model.append(input_promo)

In [ ]:
input_year = Input(shape=(1,))
embedding_4 = Embedding(3,2,name='year_embedding')(input_year)
output_year = Reshape(target_shape=(2,))(embedding_4)
# input_model.append(input_year)

In [ ]:
input_month = Input(shape=(1,))
embedding_5 = Embedding(12,6, name='month_embedding')(input_month)
output_month = Reshape(target_shape=(6,))(embedding_5)
# input_model.append(input_month)

In [ ]:
input_day = Input(shape=(1,))
embedding_6 = Embedding(31,10, name='day_embedding')(input_day)
output_day = Reshape(target_shape=(10,))(embedding_6)
# input_model.append(input_day)

In [ ]:
input_germanstate = Input(shape=(1,))
embedding_7 = Embedding(12,6, name='state_embedding')(input_germanstate)
output_germanstate = Reshape(target_shape=(6,))(embedding_7)
# input_model.append(input_germanstate)

In [ ]:
input_model = [input_store, input_dow, input_promo,
                input_year, input_month, input_day, input_germanstate] 

output_embeddings = [output_store, output_dow, output_promo,
                      output_year, output_month, output_day, output_germanstate] #do this for your output embeddings

output_model = Concatenate()(output_embeddings) # then provide the concatenate function

In [ ]:
output_model = Dense(1000, kernel_initializer='uniform')(output_model)
output_model = Activation('relu')(output_model)

output_model = Dense(500, kernel_initializer='uniform')(output_model)
output_model = Activation('relu')(output_model)

output_model = Dense(1)(output_model)
output_model = Activation('sigmoid')(output_model)

In [ ]:
#Define the embedding NN model
#your code here

modelNN_emb = KerasModel(inputs = input_model, outputs = output_model)
modelNN_emb.compile(optimizer='adam',loss='mean_absolute_error')

In [ ]:
#we will rescale our y_train for the model
#the reason for this is mentioned in the paper in the same section 
max_log_y = max(np.max(np.log(y_train)), np.max(np.log(y_val)))
fitting_ytr = np.log(y_train) / max_log_y

In [ ]:
#do the same for y_val as well
fitting_yval = np.log(y_val) / max_log_y

Now that we have built the model, we need to ensure that the input passed to the model is also in the same format - hence we will define a function split the features we are performing embeddings on

In [ ]:
#split the features
# year, month, day, Store, DoW, Promo, Open, State
def split_features(X):
    X_list = []

    store_index = X[:, [3]]
    X_list.append(store_index)

    day_of_week = X[:, [4]]
    X_list.append(day_of_week)

    promo = X[:, [5]]
    X_list.append(promo)

    year = X[:, [0]]
    X_list.append(year)

    month = X[:, [1]]
    X_list.append(month)

    day = X[:, [2]]
    X_list.append(day)

    State = X[:, [7]]
    X_list.append(State)

    return X_list

In [ ]:
def preprocessing(X):
    X_list = split_features(X)
    return X_list

1. Add a ModelCheckpoint to save the weights 

2. Fit the model on the `preprocessing(X_train)` and on the fitting_y with 10 epochs and batch_size as 128 
  * add in callback here for ModelCheckPoint and also provide validation data here. 
  * ModelCheckPoint will be used to save the weights for the model, we will use these weights for each embedding layer( you can read more about model check point [here](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint))

In [ ]:
modelNN_emb.checkpointer = ModelCheckpoint(filepath="/content/drive/MyDrive/ML2/best_model_weights.hdf5", verbose=1, save_best_only=True)
modelNN_emb.fit(preprocessing(X_train), fitting_ytr, epochs = 10, batch_size = 128, validation_data= (preprocessing(X_val),fitting_yval), callbacks=[modelNN_emb.checkpointer])

Epoch 1/10
1563/1563 [==============================] - 13s 6ms/step - loss: 0.0248 - val_loss: 0.0100

Epoch 00001: val_loss improved from inf to 0.00997, saving model to /content/drive/MyDrive/ML2/best_model_weights.hdf5
Epoch 2/10
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0097 - val_loss: 0.0089

Epoch 00002: val_loss improved from 0.00997 to 0.00886, saving model to /content/drive/MyDrive/ML2/best_model_weights.hdf5
Epoch 3/10
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0086 - val_loss: 0.0088

Epoch 00003: val_loss improved from 0.00886 to 0.00879, saving model to /content/drive/MyDrive/ML2/best_model_weights.hdf5
Epoch 4/10
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0080 - val_loss: 0.0078

Epoch 00004: val_loss improved from 0.00879 to 0.00781, saving model to /content/drive/MyDrive/ML2/best_model_weights.hdf5
Epoch 5/10
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0076 - val_loss: 0.00

Now we will save the embeddings from the model defined above:
check if save_embeddings is set to True first, if yes store the following:
  1. store_embedding
  2. dow_embedding
  3. year_embedding
  4. month_embedding
  5. day_embedding
  6. state_embedding


Save this entire embeddings into the pickle file - **saved_embeddings_fname - embeddings.pickle**

In [ ]:
if save_embeddings:
    store_embedding = modelNN_emb.get_layer('store_embedding').get_weights()[0]
    dow_embedding = modelNN_emb.get_layer('dow_embedding').get_weights()[0]
    year_embedding = modelNN_emb.get_layer('year_embedding').get_weights()[0]
    month_embedding = modelNN_emb.get_layer('month_embedding').get_weights()[0]
    day_embedding = modelNN_emb.get_layer('day_embedding').get_weights()[0]
    german_states_embedding = modelNN_emb.get_layer('state_embedding').get_weights()[0]
    with open(saved_embeddings_fname, 'wb') as f:
        pickle.dump([store_embedding, dow_embedding, year_embedding,
                     month_embedding, day_embedding, german_states_embedding], f, -1)

# You are done with Part 3!! 
Remember to save the embeddings.pickle - this will be used in Part 4!